In [1]:
import pandas as pd
import requests
import zipfile
import io
import os

# import polars as pl

In [2]:
CWD = os.path.dirname(os.getcwd())
INPUT = os.path.join(CWD, "input")
TMP = os.path.join(CWD, "tmp")

In [4]:
PIRLS_ASSETS_URLS = {
    "data": "https://pirls2021.org/data/downloads/P21_Data_SAS.zip",
    "item_information": "https://pirls2021.org/data/downloads/P21_ItemInformation.xlsx",
    "codebooks": "https://pirls2021.org/data/downloads/P21_Codebooks.zip",
}

In [4]:
if not os.path.exists(INPUT):
    os.mkdir(INPUT)

if not os.path.exists(TMP):
    os.mkdir(TMP)

In [7]:
r = requests.get(PIRLS_ASSETS_URLS["data"])
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(INPUT)

In [43]:
r = requests.get(PIRLS_ASSETS_URLS["codebooks"])
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(INPUT)

In [10]:
r = requests.get(PIRLS_ASSETS_URLS["item_information"], stream=True)
with open(f"{INPUT}/item_information.xlsx", "wb") as file:
    file.write(r.content)

In [28]:
for file in os.listdir(f"{INPUT}/Data"):
    parquet_filename = file.replace(".sas7bdat", ".parquet")
    parquet_output_filename = f"{TMP}/data/{parquet_filename}"

    pd.read_sas(f"{INPUT}/Data/{file}").to_parquet(parquet_output_filename)

In [297]:
PIRLS_TABLES_DESC = {
    "acg": "school_context",
    "asa": "student_achievement",
    "asg": "student_context",
    "ash": "home_context",
    "asr": "within_country_scoring_reliability",
    "ast": "student_teacher_link",
    "atg": "teacher_context",
}

In [5]:
def read_pirls_files():
    files = os.listdir(f"{TMP}/data")

    result = dict(acg=[], asa=[], asg=[], ash=[], asr=[], ast=[], atg=[])

    for file in files:
        is_normal_pirls = file.endswith("r5.parquet")
        table = file[0:3]
        df = pd.read_parquet(f"{TMP}/data/{file}")

        df["pirls_type"] = "Normal" if is_normal_pirls else "Bridge"

        result[table].append(df)

    return {table: pd.concat(result[table], ignore_index=True) for table in result.keys()}

In [ ]:
dfs = read_pirls_files()

# Gerar arquitetura

In [432]:
RENAMES = {
    "asa": {
        "IDCNTRY": "country_id_iso3",
        "IDPOP": "population_id",
        "IDGRADER": "standardized_grade_id",
        "IDGRADE": "grade_id",
        "WAVE": "student_wave_indicator",
        "IDSCHOOL": "school_id",
        "IDCLASS": "class_id",
        "IDSTUD": "student_id",
        "ITSEX": "sex_student",
        "ITADMINI": "test_administrator_position",
        "ITLANG_SA": "language_student_achievement_test",
        "LCID_SA": "locale_student_test_id",
        "IDBOOK": "booklet_id",
        "ITDEV": "administration_device",
        "ASDAGE": "student_age",
        "ILRELIAB": "reliability_coding_status",
        "HOUWGT": "house_weight",
        "TOTWGT": "total_student_weight",
        "SENWGT": "senate_weight",
        "WGTADJ1": "school_weight_adjustment",
        "WGTADJ2": "class_weight_adjustment",
        "WGTADJ3": "student_weight_adjustment",
        "WGTFAC1": "school_weight_factor",
        "WGTFAC2": "class_weight_factor",
        "WGTFAC3": "student_weight_factor",
        "JKREP": "jackknife_replicate_code",
        "JKZONE": "jackknife_zone",
        "SCOPE": "scope",
        "VERSION": "version",
    },
    "asr": {
        "IDCNTRY": "country_id_iso3",
        "IDPOP": "population_id",
        "IDGRADER": "standardized_grade_id",
        "IDGRADE": "grade_id",
        "WAVE": "student_wave_indicator",
        "IDSCHOOL": "school_id",
        "IDCLASS": "class_id",
        "IDSTUD": "student_id",
        "IDBOOK": "booklet_id",
        "SCOPE": "scope",
        "VERSION": "version",
    },
    "asg": {
        "IDCNTRY": "country_id_iso3",
        "IDPOP": "population_id",
        "IDGRADER": "standardized_grade_id",
        "IDGRADE": "grade_id",
        "WAVE": "student_wave_indicator",
        "IDSCHOOL": "school_id",
        "IDCLASS": "class_id",
        "IDSTUD": "student_id",
        "ITSEX": "sex_student",
        "ITADMINI": "test_administrator_position",
        "LCID_SA": "locale_student_test_id",
        "ITLANG_SA": "language_student_achievement_test",
        "ITLANG_SQ": "language_student_achievement_questionnaire",
        "LCID_SQ": "locale_student_questionnaire_id",
        "IDBOOK": "booklet_id",
        "SCOPE": "scope",
        "VERSION": "version",
    },
    "ash": {
        "IDCNTRY": "country_id_iso3",
        "IDPOP": "population_id",
        "IDGRADER": "standardized_grade_id",
        "IDGRADE": "grade_id",
        "WAVE": "student_wave_indicator",
        "IDSCHOOL": "school_id",
        "IDCLASS": "class_id",
        "IDSTUD": "student_id",
        "ITLANG_HQ": "language_home_questionnaire",
        "LCID_HQ": "locale_student_home_questionnaire_id",
        "SCOPE": "scope",
        "VERSION": "version",
    },
    "acg": {
        "IDCNTRY": "country_id_iso3",
        "IDPOP": "population_id",
        "IDGRADER": "standardized_grade_id",
        "IDGRADE": "grade_id",
        "IDSCHOOL": "school_id",
        "ITLANG_CQ": "language_school_questionnaire",
        "LCID_CQ": "locale_school_questionnaire_id",
        "SCHWGT": "school_level_weight",
        "STOTWGTU": "sum_student_weights",
        "WGTADJ1": "school_weight_adjustment",
        "WGTFAC1": "school_weight_factor",
        "JKCREP": "replicate_code",
        "JKCZONE": "zone_code",
        "SCOPE": "scope",
        "VERSION": "version",
    },
    "atg": {
        "IDCNTRY": "country_id_iso3",
        "IDPOP": "population_id",
        "IDGRADER": "standardized_grade_id",
        "IDGRADE": "grade_id",
        "IDSCHOOL": "school_id",
        "IDTEACH": "teacher_id",
        "IDLINK": "teacher_link_number",
        "IDTEALIN": "teacher_link_id",
        "ITLANG_TQ": "language_teacher_questionnaire",
        "LCID_TQ": "locale_teacher_questionnaire_id",
        # "isDummy": "is_dummy",
        "SCOPE": "scope",
        "VERSION": "version",
    },
    "ast": {
        "IDCNTRY": "country_id_iso3",
        "IDPOP": "population_id",
        "IDGRADER": "standardized_grade_id",
        "IDGRADE": "grade_id",
        "WAVE": "student_wave_indicator",
        "IDSCHOOL": "school_id",
        "IDCLASS": "class_id",
        "IDSTUD": "student_id",
        "IDTEACH": "teacher_id",
        "IDLINK": "teacher_link_number",
        "IDTEALIN": "teacher_link_id",
        "IDBOOK": "booklet_id",
        "IDSUBJ": "subject_id",
        "NTEACH": "number_teachers",
        "TCHWGT": "weight_teacher",
        "JKREP": "jackknife_replicate_code",
        "JKZONE": "jackknife_zone",
        "SCOPE": "scope",
        "VERSION": "version",
    },
}

In [433]:
def gen_labels_from_context_questionnaires():
    return {
        "asg": {
            "ASBG01": "Which of these describes you?",
            "ASBG02A": "When were you born? Month",
            "ASBG02B": "When were you born? Year",
            "ASBG03": "How often do you speak <language of test> at home?",
            "ASBG04": "About how many books are there in your home? (Do not count magazines, newspapers, or your school books.)",
            "ASBG05A": "Do you have any of these things at your home? A shared computer or tablet that you can use",
            "ASBG05B": "Do you have any of these things at your home? Study desk/table for your use",
            "ASBG05C": "Do you have any of these things at your home? Your own room",
            "ASBG05D": "Do you have any of these things at your home? Access to the internet",
            "ASBG05E": "Do you have any of these things at your home? A shared smartphone that you can use",
            "ASBG05F": "Do you have any of these things at your home? Your own computer or Tablet",
            "ASBG05G": "Do you have any of these things at your home? Your own smartphone",
            "ASBG05H": "Do you have any of these things at your home? <country-specific indicator of wealth>",
            "ASBG05I": "Do you have any of these things at your home? <country-specific indicator of wealth>",
            "ASBG05J": "Do you have any of these things at your home? <country-specific indicator of wealth>",
            "ASBG05K": "Do you have any of these things at your home? <country-specific indicator of wealth>",
            "ASBG06": "About how often are you absent from school?",
            "ASBG07A": "How often do you feel this way when you arrive at school? I feel tired",
            "ASBG07B": "How often do you feel this way when you arrive at school? I feel hungry",
            "ASBG08A": "How much time do you spend using a computer, tablet, or smartphone to do these activities for your schoolwork on a normal school day? Finding and reading information",
            "ASBG08B": "How much time do you spend using a computer, tablet, or smartphone to do these activities for your schoolwork on a normal school day? Preparing reports and presentations",
            "ASBG09A": "How much do you agree with these statements about using computers, tablets, or smartphones? I am good at using a computer or tablet",
            "ASBG09B": "How much do you agree with these statements about using computers, tablets, or smartphones? I am good at typing",
            "ASBG09C": "How much do you agree with these statements about using computers, tablets, or smartphones? It is easy for me to find information on the Internet",
            "ASBG09D": "How much do you agree with these statements about using computers, tablets, or smartphones? I know how to create written stories or reports",
            "ASBG09E": "How much do you agree with these statements about using computers, tablets, or smartphones? I know how to create presentations",
            "ASBG09F": "How much do you agree with these statements about using computers, tablets, or smartphones? I can recognize a website that is useful to me",
            "ASBG09G": "How much do you agree with these statements about using computers, tablets, or smartphones? I can tell if a website is trustworthy",
            "ASBG09H": "How much do you agree with these statements about using computers, tablets, or smartphones? I know how to make and share a video",
            "ASBG10A": "What do you think about your school? Tell how much you agree with these statements. I like being in school",
            "ASBG10B": "What do you think about your school? Tell how much you agree with these statements. I feel safe when I am at school",
            "ASBG10C": "What do you think about your school? Tell how much you agree with These statements. I feel like I belong at this school",
            "ASBG10D": "What do you think about your school? Tell how much you agree with these statements. Teachers at my school are fair to me",
            "ASBG10E": "What do you think about your school? Tell how much you agree with these statements. I am proud to go to this school",
            "ASBG10F": "What do you think about your school? Tell how much you agree with these statements. I have friends at this school",
            "ASBG11A": "During this year, how often have other students from your school done any of the following things to you, including through texting or the internet? Made fun of me or called me names",
            "ASBG11B": "During this year, how often have other students from your school done any of the following things to you, including through texting or the internet? Left me out of their games or activities",
            "ASBG11C": "During this year, how often have other students from your school done any of the following things to you, including through texting or the internet? Spread lies about me",
            "ASBG11D": "During this year, how often have other students from your school done any of the following things to you, including through texting or the internet? Stole something from me",
            "ASBG11E": "During this year, how often have other students from your school done any of the following things to you, including through texting or the internet? Damaged something of mine on purpose",
            "ASBG11F": "During this year, how often have other students from your school done any of the following things to you, including through texting or the internet? Hit or hurt me (e.g., shoving, kicking, hitting)",
            "ASBG11G": "During this year, how often have other students from your school done any of the following things to you, including through texting or the internet? Made me do things I didn't want to do",
            "ASBG11H": "During this year, how often have other students from your school done any of the following things to you, including through texting or the internet? Sent me nasty or hurtful messages online",
            "ASBG11I": "During this year, how often have other students from your school done any of the following things to you, including through texting or the internet? Shared nasty or hurtful information about me online",
            "ASBG11J": "During this year, how often have other students from your school done any of the following things to you, including through texting or the internet? Threatened me",
            "ASBR01A": "Think about the reading you do for school. How much do you agree with these statements about your reading lessons? I like what I read about in school",
            "ASBR01B": "Think about the reading you do for school. How much do you agree with these statements about your reading lessons? My teacher gives me interesting things to read",
            "ASBR01C": "Think about the reading you do for school. How much do you agree with these statements about your reading lessons? I know what my teacher expects me to do",
            "ASBR01D": "Think about the reading you do for school. How much do you agree with these statements about your reading lessons? My teacher is easy to understand",
            "ASBR01E": "Think about the reading you do for school. How much do you agree with these statements about your reading lessons? I am interested in what my teacher says",
            "ASBR01F": "Think about the reading you do for school. How much do you agree with these statements about your reading lessons? My teacher encourages me to say what I think about what I have read",
            "ASBR01G": "Think about the reading you do for school. How much do you agree with these statements about your reading lessons? My teacher lets me show what I have learned",
            "ASBR01H": "Think about the reading you do for school. How much do you agree with these statements about your reading lessons? My teacher does a variety of things to help us learn",
            "ASBR01I": "Think about the reading you do for school. How much do you agree with these statements about your reading lessons? My teacher tells me how to do better when I make a mistake",
            "ASBR02A": "How often do these things happen during your reading lessons? Students don't listen to what the teacher says",
            "ASBR02B": "How often do these things happen during your reading lessons? There is too much noise for students to work well",
            "ASBR02C": "How often do these things happen during your reading lessons? My teacher has to wait a long time for students to be quiet",
            "ASBR02D": "How often do these things happen during your reading lessons? Students interrupt the teacher",
            "ASBR02E": "How often do these things happen during your reading lessons? My teacher has to keep telling us to follow the classroom rules",
            "ASBR03A": "In school, how often do these things happen? I read silently on my own",
            "ASBR03B": "In school, how often do these things happen? I read things that I choose myself",
            "ASBR03C": "In school, how often do these things happen? My teacher asks us in class to talk about what we have read",
            "ASBR04": "How often do you borrow books (including ebooks) from your <school or local library>?",
            "ASBR05": "How much time do you spend reading outside of school on a normal school day?",
            "ASBR06A": "How often do you do these things outside of school? I read for fun",
            "ASBR06B": "How often do you do these things outside of school? I read to find out about things I want to learn",
            "ASBR07A": "What do you think about reading? Tell how much you agree with each of these statements. I like talking about what I read with other people",
            "ASBR07B": "What do you think about reading? Tell how much you agree with each of these statements. I would be happy if someone gave me a book as a present",
            "ASBR07C": "What do you think about reading? Tell how much you agree with each of these statements. I think reading is boring",
            "ASBR07D": "What do you think about reading? Tell how much you agree with each of these statements. I would like to have more time for reading",
            "ASBR07E": "What do you think about reading? Tell how much you agree with each of these statements. I enjoy reading",
            "ASBR07F": "What do you think about reading? Tell how much you agree with each of these statements. I learn a lot from reading",
            "ASBR07G": "What do you think about reading? Tell how much you agree with each of these statements. I like to read things that make me think",
            "ASBR07H": "What do you think about reading? Tell how much you agree with each of these statements. I like it when a book helps me imagine other worlds",
            "ASBR08A": "How well do you read? Tell how much you agree with each of these statements. I usually do well in reading",
            "ASBR08B": "How well do you read? Tell how much you agree with each of these statements. Reading is easy for me",
            "ASBR08C": "How well do you read? Tell how much you agree with each of these statements. I have trouble reading stories with difficult words",
            "ASBR08D": "How well do you read? Tell how much you agree with each of these statements. Reading is harder for me than for many of my classmates",
            "ASBR08E": "How well do you read? Tell how much you agree with each of these statements. Reading is harder for me than any other subject",
            "ASBR08F": "How well do you read? Tell how much you agree with each of these statements. I am just not good at reading",
        },
        "ash": {
            "ASBH01A": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Read books",
            "ASBH01B": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Tell stories",
            "ASBH01C": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Sing songs",
            "ASBH01D": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Play with alphabet toys (e.g., blocks with letters of the alphabet)",
            "ASBH01E": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Talk about things you had done",
            "ASBH01F": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Talk about what you had read",
            "ASBH01G": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Play word games",
            "ASBH01H": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Write letters or words",
            "ASBH01I": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Read aloud signs and labels",
            "ASBH01J": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Say counting rhymes or sing counting songs",
            "ASBH01K": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Play with number toys (e.g., blocks with numbers)",
            "ASBH01L": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Count different things",
            "ASBH01M": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Play games involving shapes (e.g., shape sorting toys, puzzles)",
            "ASBH01N": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Play with building blocks or construction toys",
            "ASBH01O": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Play board or card games",
            "ASBH01P": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Write numbers",
            "ASBH01Q": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Draw shapes",
            "ASBH01R": "Before your child began primary/elementary school, how often did you or someone else in your home do the following activities with him or her? Measure or weigh things (e.g., when cooking)",
            "ASBH02A": "Was your child born in <country>?",
            "ASBH02B": "If No, How old was your child when he/she came to <country>?",
            "ASBH03A": "What language did your child speak before he/she began school? <language of test>",
            "ASBH03B": "What language did your child speak before he/she began school? <country-specific>",
            "ASBH03C": "What language did your child speak before he/she began school? <country-specific>",
            "ASBH03D": "What language did your child speak before he/she began school? <country-specific>",
            "ASBH03E": "What language did your child speak before he/she began school? <country-specific>",
            "ASBH03F": "What language did your child speak before he/she began school? Other",
            "ASBH04": "How often does your child speak <language of test> at home?",
            "ASBH05AA": "Did your child attend the following before <first grade>? Early childhood educational program or center for children under age 3",
            "ASBH05AB": "Did your child attend the following before <first grade>? Pre-primary educational program for children age 3 or older, including <Kindergarten>",
            "ASBH05B": "Approximately, how long was your child in these programs altogether?",
            "ASBH06": "How old was your child when he/she began the <first grade> of primary/elementary school?",
            "ASBH07A": "How well could your child do the following when he/she began the <first grade> of primary/elementary school? Recognize most of the letters of the alphabet",
            "ASBH07B": "How well could your child do the following when he/she began the <first grade> of primary/elementary school? Read some words",
            "ASBH07C": "How well could your child do the following when he/she began the <first grade> of primary/elementary school? Read sentences",
            "ASBH07D": "How well could your child do the following when he/she began the <first grade> of primary/elementary school? Read a story",
            "ASBH07E": "How well could your child do the following when he/she began the <first grade> of primary/elementary school? Write letters of the alphabet",
            "ASBH07F": "How well could your child do the following when he/she began the <first grade> of primary/elementary school? Write his/her name",
            "ASBH07G": "How well could your child do the following when he/she began the <first grade> of primary/elementary school? Write words other than his/her name",
            "ASBH08A": "What do you think of your child’s school? My child’s school does a good job including me in my child’s education",
            "ASBH08B": "What do you think of your child’s school? My child’s school provides a safe environment",
            "ASBH08C": "What do you think of your child’s school? My child’s school cares about my child’s progress in school",
            "ASBH08D": "What do you think of your child’s school? My child’s school does a good job informing me of his/her progress",
            "ASBH08E": "What do you think of your child’s school? My child’s school promotes high academic standards",
            "ASBH08F": "What do you think of your child’s school? My child’s school does a good job in helping him/her become better in reading",
            "ASBH09": "In a typical week, how much time do you usually spend reading for yourself at home, including books, magazines, newspapers, and materials for work (in print or digital media)?",
            "ASBH10": "When you are at home, how often do you read for your own enjoyment?",
            "ASBH11A": "Please indicate how much you agree with the following statements about reading. I read only if I have to",
            "ASBH11B": "Please indicate how much you agree with the following statements about reading. I like talking about what I read with other people",
            "ASBH11C": "Please indicate how much you agree with the following statements about reading. I like to spend my spare time reading",
            "ASBH11D": "Please indicate how much you agree with the following statements about reading. I read only if I need information",
            "ASBH11E": "Please indicate how much you agree with the following statements about reading. Reading is an important activity in my home",
            "ASBH11F": "Please indicate how much you agree with the following statements about reading. I would like to have more time for reading",
            "ASBH11G": "Please indicate how much you agree with the following statements about reading. I enjoy reading",
            "ASBH11H": "Please indicate how much you agree with the following statements about reading. Reading is one of my favorite hobbies",
            "ASBH12": "About how many books are there in your home? (Do not count ebooks, magazines, newspapers, or children’s books.)",
            "ASBH13": "About how many children’s books are there in your home? (Do not count children’s ebooks, magazines, or school books.)",
            "ASBH14A": "Do you have any of these things in your home? Access to the internet",
            "ASBH14B": "Do you have any of these things in your home? A computer, tablet, or e-reader",
            "ASBH14C": "Do you have any of these things in your home? A smartphone",
            "ASBH15A": "What is the highest level of education completed by the child/s <parents/guardians>? <Parent/Guardian A>",
            "ASBH15B": "What is the highest level of education completed by the child/s <parents/guardians>? <Parent/Guardian B>",
            "ASBH16": "How far in his/her education do you expect your child to go?",
            "ASBH17A": "What kind of work do the child's <parents/guardians> do for their main jobs? <Parent/Guardian A>",
            "ASBH17B": "What kind of work do the child's <parents/guardians> do for their main jobs? <Parent/Guardian B>",
            "ASBH18AA": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian A> <language of test>",
            "ASBH18AB": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian B> <language of test>",
            "ASBH18BA": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian A> <country-specific>",
            "ASBH18BB": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian B> <country-specific>",
            "ASBH18CA": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian A> <country-specific>",
            "ASBH18CB": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian B> <country-specific>",
            "ASBH18DA": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian A> <country-specific>",
            "ASBH18DB": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian B> <country-specific>",
            "ASBH18EA": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian A> <country-specific>",
            "ASBH18EB": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian B> <country-specific>",
            "ASBH18FA": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian A> <Other>",
            "ASBH18FB": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian B> <Other>",
            "ASBH18GA": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian A> <Not applicable>",
            "ASBH18GB": "Do the child's <parents/guardians> talk with the child in the following languages? <Parent/Guardian B> <Not Applicable>",
            "ASBH19": "Did your child stay at home any time because of the COVID-19 pandemic?",
            "ASBH20A": "When your child was not in school, how did your child's school engage with your child's home-based learning? Reading assignments",
            "ASBH20B": "When your child was not in school, how did your child's school engage with your child's home-based learning? Online activities",
            "ASBH20C": "When your child was not in school, how did your child's school engage with your child's home-based learning? Printed learning materials",
            "ASBH21A": "Did you provide additional educational resources for your child during the pandemic? Books",
            "ASBH21B": "Did you provide additional educational resources for your child during the pandemic? Digital devices",
            "ASBH21C": "Did you provide additional educational resources for your child during the pandemic? Digitally-based learning activities",
            "ASBH21D": "Did you provide additional educational resources for your child during the pandemic? Online instruction or tutoring",
            "ASBH22": "Do you think your child's learning progress has been adversely affected?",
        },
        "acg": {
            "ACBG01": "What is the total enrollment of students in your school as of <first day of month PIRLS testing begins, 2021>?",
            "ACBG02": "What is the enrollment of <fourth grade> students in your school as of <first day of month PIRLS testing begins, 2021>?",
            "ACBG03A": "Approximately what percentage of students in your school have the following backgrounds? Come from economically disadvantaged homes",
            "ACBG03B": "Approximately what percentage of students in your school have the following backgrounds? Come from economically affluent homes",
            "ACBG04": "Approximately what percentage of students in your school have <language of test> as their native language?",
            "ACBG05A": "How many people live in the city, town, or area where your school is located?",
            "ACBG05B": "Which best describes the immediate area in which your school is located?",
            "ACBG06A": "For the <fourth grade> students in your school: How many days per year is your school open for instruction?",
            "ACBG06B": "For the <fourth grade> students in your school: What is the total instructional time, excluding breaks, in a typical day? (minutes)",
            "ACBG06C": "For the <fourth grade> students in your school: In one calendar week, how many days is the school open for instruction?",
            "ACBG07A": "Does your school have a school library?",
            "ACBG07B": "Approximately how many books (print) with different titles does your school library have (exclude magazines and periodicals)?",
            "ACBG07C": "Can students borrow print or digital materials from the library to take home?",
            "ACBG08": "Does the school provide access to digital learning resources (e.g., books, videos)?",
            "ACBG09": "How many computers (including tablets) does your school have for use by <fourth grade> students?",
            "ACBG10AA": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? General School Resources: Instructional materials (e.g., textbooks)",
            "ACBG10AB": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? General School Resources: Supplies (e.g., papers, pencils, materials)",
            "ACBG10AC": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? General School Resources: School buildings and grounds",
            "ACBG10AD": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? General School Resources: Heating/cooling and lighting systems",
            "ACBG10AE": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? General School Resources: Instructional space (e.g., classrooms)",
            "ACBG10AF": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? General School Resources: Technologically competent staff",
            "ACBG10AG": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? General School Resources: Technology and media resources to support teaching",
            "ACBG10AH": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? General School Resources: Technology and media resources to support student learning and expression",
            "ACBG10AI": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? General School Resources: Resources for students with disabilities",
            "ACBG10AJ": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? General School Resources: Internet connection",
            "ACBG10BA": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? Resources for Reading Instruction: Teachers with a specialization in reading",
            "ACBG10BB": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? Resources for Reading Instruction: Computer software/applications for reading instruction",
            "ACBG10BC": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? Resources for Reading Instruction: Library resources (books, ebooks, magazines, etc.)",
            "ACBG10BD": "How much is your school’s capacity to provide instruction affected by a shortage or inadequacy of the following? Resources for Reading Instruction: Instructional materials for reading",
            "ACBG11A": "How would you characterize each of the following within your school? Teachers’ understanding of the school’s curricular goals",
            "ACBG11B": "How would you characterize each of the following within your school? Teachers’ degree of success in implementing the school’s curriculum",
            "ACBG11C": "How would you characterize each of the following within your school? Teachers’ expectations for student achievement",
            "ACBG11D": "How would you characterize each of the following within your school? Teachers’ ability to inspire students",
            "ACBG11E": "How would you characterize each of the following within your school? Collaboration between school leadership and teachers to plan instruction",
            "ACBG11F": "How would you characterize each of the following within your school? Parental involvement in school activities",
            "ACBG11G": "How would you characterize each of the following within your school? Parental commitment to ensure that students are ready to learn",
            "ACBG11H": "How would you characterize each of the following within your school? Parental expectations for student achievement",
            "ACBG11I": "How would you characterize each of the following within your school? Parental support for student achievement",
            "ACBG11J": "How would you characterize each of the following within your school? Students’ desire to do well in school",
            "ACBG11K": "How would you characterize each of the following within your school? Students’ ability to reach school’s academic goals",
            "ACBG11L": "How would you characterize each of the following within your school? Students’ respect for classmates who excel academically",
            "ACBG12A": "To what degree is each of the following a problem among <fourth grade> students in your school? Arriving late at school",
            "ACBG12B": "To what degree is each of the following a problem among <fourth grade> students in your school? Absenteeism (i.e., unjustified absences)",
            "ACBG12C": "To what degree is each of the following a problem among <fourth grade> students in your school? Classroom disturbance",
            "ACBG12D": "To what degree is each of the following a problem among <fourth grade> students in your school? Cheating",
            "ACBG12E": "To what degree is each of the following a problem among <fourth grade> students in your school? Profanity",
            "ACBG12F": "To what degree is each of the following a problem among <fourth grade> students in your school? Vandalism",
            "ACBG12G": "To what degree is each of the following a problem among <fourth grade> students in your school? Theft",
            "ACBG12H": "To what degree is each of the following a problem among <fourth grade> students in your school? Intimidation or verbal abuse among students (including texting, emailing, etc.)",
            "ACBG12I": "To what degree is each of the following a problem among <fourth grade> students in your school? Physical fights among students",
            "ACBG12J": "To what degree is each of the following a problem among <fourth grade> students in your school? Intimidation or verbal abuse of teachers or staff (including texting, emailing, etc.)",
            "ACBG13": "About how many of the students in your school have basic literacy skills (e.g., can write letters of the alphabet, write sentences) when they begin the <first grade> or primary/elementary school?",
            "ACBG14A": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Knowing letters of the alphabet",
            "ACBG14B": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Knowing letter-sound relationships",
            "ACBG14C": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Reading words",
            "ACBG14D": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Reading isolated sentences",
            "ACBG14E": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Reading connected text",
            "ACBG14F": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Locating information within the text",
            "ACBG14G": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Identifying the main idea of a text",
            "ACBG14H": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Explaining or supporting understanding of a text",
            "ACBG14I": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Comparing a text with personal experience",
            "ACBG14J": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Comparing different texts",
            "ACBG14K": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Making predictions about what will happen next in a text",
            "ACBG14L": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Making generalizations and drawing inferences based on a text",
            "ACBG14M": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Evaluating and critiquing the style or structure of a text",
            "ACBG14N": "At which grade do the following reading skills and strategies first receive a major emphasis in instruction in your school? Determining the author’s perspective or intention",
            "ACBG15": "By the end of this school year, how many years will you have been a principal altogether?",
            "ACBG16": "By the end of this school year, how many years will you have been a principal at this school?",
            "ACBG17": "What is the highest level of formal education you have completed?",
            "ACBG18A": "Do you hold any of the following professional qualifications in educational leadership? <Certificate or license>",
            "ACBG18B": "Do you hold any of the following professional qualifications in educational leadership? <Master's or equivalent level—ISCED Level 7>",
            "ACBG18C": "Do you hold any of the following professional qualifications in educational leadership? <Doctor or equivalent level—ISCED Level 8>",
            "ACBG19": "Please estimate the number of weeks during the current academic year where normal primary school operations have been affected by the COVID-19 pandemic.",
            "ACBG20": "Does your school provide remote instruction or distance learning resources for primary grades when normal school operations are affected by the COVID-19 pandemic?",
            "ACBG21A": "When normal primary school operations are affected by the COVID- 19 pandemic, does your school support remote learning through the following? Delivery of printed learning materials to students",
            "ACBG21B": "When normal primary school operations are affected by the COVID- 19 pandemic, does your school support remote learning through the following? Internet-based resources for students",
            "ACBG21C": "When normal primary school operations are affected by the COVID- 19 pandemic, does your school support remote learning through the following? Access to digital devices for students",
            "ACBG21D": "When normal primary school operations are affected by the COVID- 19 pandemic, does your school support remote learning through the following? Recommendations for teachers about how to provide online instruction",
            "ACBG21E": "When normal primary school operations are affected by the COVID- 19 pandemic, does your school support remote learning through the following? Technical support for teachers",
            "ACBG21F": "When normal primary school operations are affected by the COVID- 19 pandemic, does your school support remote learning through the following? Access to digital devices for teachers",
        },
        "atg": {
            "ATBG01": "By the end of this school year, how many years will you have been teaching altogether?",
            "ATBG02": "Which of these describes you?",
            "ATBG03": "How old are you?",
            "ATBG04": "What is the highest level of formal education you have completed?",
            "ATBG05AA": "During your <post-secondary> education, what was your major or main area(s) of study? Education—Primary/Elementary",
            "ATBG05AB": "During your <post-secondary> education, what was your major or main area(s) of study? Education—Secondary",
            "ATBG05AC": "During your <post-secondary> education, what was your major or main area(s) of study? <language of test>",
            "ATBG05AD": "During your <post-secondary> education, what was your major or main area(s) of study? Other",
            "ATBG05BA": "As part of your formal education and/or training, to what extent did you study the following areas? <language of test>",
            "ATBG05BB": "As part of your formal education and/or training, to what extent did you study the following areas? Literature",
            "ATBG05BC": "As part of your formal education and/or training, to what extent did you study the following areas? Pedagogy/teaching reading",
            "ATBG05BD": "As part of your formal education and/or training, to what extent did you study the following areas? Educational psychology",
            "ATBG05BE": "As part of your formal education and/or training, to what extent did you study the following areas? Learning support",
            "ATBG05BF": "As part of your formal education and/or training, to what extent did you study the following areas? Reading theory",
            "ATBG05BG": "As part of your formal education and/or training, to what extent did you study the following areas? Special education",
            "ATBG05BH": "As part of your formal education and/or training, to what extent did you study the following areas? Second language learning",
            "ATBG05BI": "As part of your formal education and/or training, to what extent did you study the following areas? Assessment methods in reading",
            "ATBG05BJ": "As part of your formal education and/or training, to what extent did you study the following areas? Early childhood education",
            "ATBG05BK": "As part of your formal education and/or training, to what extent did you study the following areas? Digital literacies",
            "ATBG06": "How often do you read for enjoyment?",
            "ATBG07AA": "In the past two years, have you participated in formal professional development in reading (e.g., workshops, seminars, lesson studies)? Teaching reading comprehension skills or strategies",
            "ATBG07BA": "How would you prioritize your need for future professional development? Teaching reading comprehension skills or strategies",
            "ATBG07AB": "In the past two years, have you participated in formal professional development in reading (e.g., workshops, seminars, lesson studies)? Integrating literacies across the curriculum",
            "ATBG07BB": "How would you prioritize your need for future professional development? Integrating literacies across the curriculum",
            "ATBG07AC": "In the past two years, have you participated in formal professional development in reading (e.g., workshops, seminars, lesson studies)? Addressing students' language needs in teaching reading",
            "ATBG07BC": "How would you prioritize your need for future professional development? Addressing students' language needs in teaching Reading",
            "ATBG07AD": "In the past two years, have you participated in formal professional development in reading (e.g., workshops, seminars, lesson studies)? Integrating technology into reading instruction",
            "ATBG07BD": "How would you prioritize your need for future professional development? Integrating technology into reading instruction",
            "ATBG07AE": "In the past two years, have you participated in formal professional development in reading (e.g., workshops, seminars, lesson studies)? Instruction related to digital literacies",
            "ATBG07BE": "How would you prioritize your need for future professional development? Instruction related to digital literacies",
            "ATBG07AF": "In the past two years, have you participated in formal professional development in reading (e.g., workshops, seminars, lesson studies)? Addressing differentiation of instruction for students' needs and Interests",
            "ATBG07BF": "How would you prioritize your need for future professional development? Addressing differentiation of instruction for students' needs and interests",
            "ATBG07AG": "In the past two years, have you participated in formal professional development in reading (e.g., workshops, seminars, lesson studies)? Assessing students' reading",
            "ATBG07BG": "How would you prioritize your need for future professional development? Assessing students' reading",
            "ATBG08A": "How helpful is each type of professional development? Workshops",
            "ATBG08B": "How helpful is each type of professional development? Seminars",
            "ATBG08C": "How helpful is each type of professional development? Access to a mentor (e.g., literacy coach)",
            "ATBG08D": "How helpful is each type of professional development? Teacher professional learning communities",
            "ATBG08E": "How helpful is each type of professional development? Online professional development",
            "ATBG09A": "To what extent are each of these a disincentive to participate in professional development? Financial costs",
            "ATBG09B": "To what extent are each of these a disincentive to participate in professional development? Time conflicts",
            "ATBG09C": "To what extent are each of these a disincentive to participate in professional development? Content is not relevant",
            "ATBG09D": "To what extent are each of these a disincentive to participate in professional development? Lack of support from school administrators",
            "ATBG10A": "How would you characterize each of the following within your school? Teachers’ understanding of the school’s curricular goals",
            "ATBG10B": "How would you characterize each of the following within your school? Teachers’ degree of success in implementing the school’s curriculum",
            "ATBG10C": "How would you characterize each of the following within your school? Teachers’ expectations for student achievement",
            "ATBG10D": "How would you characterize each of the following within your school? Teachers’ ability to inspire students",
            "ATBG10E": "How would you characterize each of the following within your school? Collaboration between school leadership and teachers to plan instruction",
            "ATBG10F": "How would you characterize each of the following within your school? Parental involvement in school activities",
            "ATBG10G": "How would you characterize each of the following within your school? Parental commitment to ensure that students are ready to learn",
            "ATBG10H": "How would you characterize each of the following within your school? Parental expectations for student achievement",
            "ATBG10I": "How would you characterize each of the following within your school? Parental support for student achievement",
            "ATBG10J": "How would you characterize each of the following within your school? Students’ desire to do well in school",
            "ATBG10K": "How would you characterize each of the following within your school? Students’ ability to reach school’s academic goals",
            "ATBG10L": "How would you characterize each of the following within your school? Students’ respect for classmates who excel academically",
            "ATBG11A": "Thinking about your current school, indicate the extent to which you agree or disagree with each of the following statements. This school is located in a safe neighborhood",
            "ATBG11B": "Thinking about your current school, indicate the extent to which you agree or disagree with each of the following statements. I feel safe at this school",
            "ATBG11C": "Thinking about your current school, indicate the extent to which you agree or disagree with each of the following statements. This school's security policies and practices are sufficient",
            "ATBG11D": "Thinking about your current school, indicate the extent to which you agree or disagree with each of the following statements. The students behave in an orderly manner",
            "ATBG11E": "Thinking about your current school, indicate the extent to which you agree or disagree with each of the following statements. The students are respectful of the teachers",
            "ATBG11F": "Thinking about your current school, indicate the extent to which you agree or disagree with each of the following statements. The students respect school property",
            "ATBG11G": "Thinking about your current school, indicate the extent to which you agree or disagree with each of the following statements. This school has clear rules about student conduct",
            "ATBG11H": "Thinking about your current school, indicate the extent to which you agree or disagree with each of the following statements. This school's rules are enforced in a fair and consistent manner",
            "ATBG11I": "Thinking about your current school, indicate the extent to which you agree or disagree with each of the following statements. The students are respectful of each other",
            "ATBG12A": "How often do you feel the following way about being a teacher? I am content with my profession as a teacher",
            "ATBG12B": "How often do you feel the following way about being a teacher? I find my work full of meaning and purpose",
            "ATBG12C": "How often do you feel the following way about being a teacher? I am enthusiastic about my job",
            "ATBG12D": "How often do you feel the following way about being a teacher? My work inspires me",
            "ATBG12E": "How often do you feel the following way about being a teacher? I am proud of the work I do",
            "ATBG12F": "How often do you feel the following way about being a teacher? I feel appreciated as a teacher",
            "ATBR01A": "How many students are in this class?",
            "ATBR01B": "How many of the students in #R1A are in <fourth grade>?",
            "ATBR02A": "How many <fourth grade> students experience difficulties understanding spoken <language of test>?",
            "ATBR02B": "How many <fourth grade> students experience difficulties in reading?",
            "ATBR03A": "In your view, to what extent do the following limit how you teach this class? Students lacking prerequisite knowledge or skills",
            "ATBR03B": "In your view, to what extent do the following limit how you teach this class? Students suffering from lack of basic nutrition",
            "ATBR03C": "In your view, to what extent do the following limit how you teach this class? Students suffering from not enough sleep",
            "ATBR03D": "In your view, to what extent do the following limit how you teach this class? Students absent from class",
            "ATBR03E": "In your view, to what extent do the following limit how you teach this class? Disruptive students",
            "ATBR03F": "In your view, to what extent do the following limit how you teach this class? Uninterested students",
            "ATBR03G": "In your view, to what extent do the following limit how you teach this class? Students with mental, emotional, or psychological impairment",
            "ATBR03H": "In your view, to what extent do the following limit how you teach this class? Students needing extra support in reading",
            "ATBR04": "In a typical week, how much time do you spend on <language of test> language instruction and/or activities with the students? (minutes per week)",
            "ATBR05": "Regardless of whether or not you have formally scheduled time for reading instruction, in a typical week about how much time do you spend on reading instruction and/or activities with the students? (minutes per week)",
            "ATBR06A": "When you have reading instruction and/or do reading activities, how often do you organize students in the following ways? I teach reading as a whole-class activity",
            "ATBR06B": "When you have reading instruction and/or do reading activities, how often do you organize students in the following ways? I create same- ability groups",
            "ATBR06C": "When you have reading instruction and/or do reading activities, how often do you organize students in the following ways? I create mixed- ability groups",
            "ATBR06D": "When you have reading instruction and/or do reading activities, how often do you organize students in the following ways? I use individualized instruction for reading",
            "ATBR06E": "When you have reading instruction and/or do reading activities, how often do you organize students in the following ways? Students work independently on an assigned goal or plan",
            "ATBR07AA": "When you have reading instruction and/or do reading activities with the students, how often do you have the students read the following types of text (in print or digitally)? Literary Reading Materials: Short stories (e.g., fables, fairy tales, action stories, science fiction, detective stories)",
            "ATBR07AB": "When you have reading instruction and/or do reading activities with the students, how often do you have the students read the following types of text (in print or digitally)? Literary Reading Materials: Longer fiction books with chapters",
            "ATBR07AC": "When you have reading instruction and/or do reading activities with the students, how often do you have the students read the following types of text (in print or digitally)? Literary Reading Materials: Plays",
            "ATBR07AD": "When you have reading instruction and/or do reading activities with the students, how often do you have the students read the following types of text (in print or digitally)? Literary Reading Materials: Poems/poetry",
            "ATBR07BA": "When you have reading instruction and/or do reading activities with the students, how often do you have the students read the following types of text (in print or digitally)? Informational Reading Materials: Nonfiction subject area books or textbooks",
            "ATBR07BB": "When you have reading instruction and/or do reading activities with the students, how often do you have the students read the following types of text (in print or digitally)? Informational Reading Materials: Longer nonfiction books with chapters",
            "ATBR07BC": "When you have reading instruction and/or do reading activities with the students, how often do you have the students read the following types of text (in print or digitally)? Informational Reading Materials: Nonfiction articles that describe and explain about things, people, events, or how things work (e.g., newspaper articles, brochures)",
            "ATBR07BD": "When you have reading instruction and/or do reading activities with the students, how often do you have the students read the following types of text (in print or digitally)? Informational Reading Materials: Non-continuous texts (e.g., diagrams, maps, illustrations, photographs, tables)",
            "ATBR08A": "When you have reading instruction and/or do reading activities with the students, how often do you do the following? Read aloud to students",
            "ATBR08B": "When you have reading instruction and/or do reading activities with the students, how often do you do the following? Ask students to read aloud",
            "ATBR08C": "When you have reading instruction and/or do reading activities with the students, how often do you do the following? Ask students to read silently on their own",
            "ATBR08D": "When you have reading instruction and/or do reading activities with the students, how often do you do the following? Teach students strategies for decoding sounds and words",
            "ATBR08E": "When you have reading instruction and/or do reading activities with the students, how often do you do the following? Teach students new vocabulary systematically",
            "ATBR08F": "When you have reading instruction and/or do reading activities with the students, how often do you do the following? Provide opportunities for students to develop fluency",
            "ATBR08G": "When you have reading instruction and/or do reading activities with the students, how often do you do the following? Teach or model skimming or scanning strategies",
            "ATBR08H": "When you have reading instruction and/or do reading activities with the students, how often do you do the following? Teach digital literacy skills (e.g., read, write, and communicate using digital tools and media)",
            "ATBR09A": "How often do you do the following in teaching reading to this class? Provide reading materials that match the students’ interests",
            "ATBR09B": "How often do you do the following in teaching reading to this class? Provide materials that are appropriate for the reading levels of individual students",
            "ATBR09C": "How often do you do the following in teaching reading to this class? Link new content to students’ prior knowledge",
            "ATBR09D": "How often do you do the following in teaching reading to this class? Encourage students to develop their understandings of the text",
            "ATBR09E": "How often do you do the following in teaching reading to this class? Encourage student discussions of texts",
            "ATBR09F": "How often do you do the following in teaching reading to this class? Encourage students to challenge the opinion expressed in the text",
            "ATBR09G": "How often do you do the following in teaching reading to this class? Encourage students to read texts with multiple perspectives",
            "ATBR09H": "How often do you do the following in teaching reading to this class? Give students time to read books of their own choosing",
            "ATBR09I": "How often do you do the following in teaching reading to this class? Give individualized feedback to each student",
            "ATBR10A": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Locate information within the text",
            "ATBR10B": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Identify the main ideas of what they have read",
            "ATBR10C": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Explain or support their understanding with text evidence",
            "ATBR10D": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Compare what they have read with experiences they have had",
            "ATBR10E": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Compare what they have read with other things they have read",
            "ATBR10F": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Make predictions about what will happen next in the text they are reading",
            "ATBR10G": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Make generalizations and draw inferences based on what they have read",
            "ATBR10H": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Evaluate and critique the style or structure of the text they have read",
            "ATBR10I": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Determine the author’s perspective or intention",
            "ATBR10J": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Self-monitor their reading (e.g., recognize when they don't understand)",
            "ATBR10K": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Determine if a website is useful for a specific purpose",
            "ATBR10L": "How often do you ask the students to do the following things to help develop reading comprehension skills or strategies? Evaluate the credibility of a website",
            "ATBR11A": "After students have read something, how often do you ask them to do the following? Write something about or in response to what they have read",
            "ATBR11B": "After students have read something, how often do you ask them to do the following? Answer oral questions about or orally summarize what they have read",
            "ATBR11C": "After students have read something, how often do you ask them to do the following? Talk with each other about what they have read",
            "ATBR11D": "After students have read something, how often do you ask them to do the following? Take a written quiz or test about what they have read",
            "ATBR11E": "After students have read something, how often do you ask them to do the following? Create a multi-model response (e.g., image, audio, text, video, performance)",
            "ATBR12A": "Do the students in this class ever have digital devices (desktop computers, laptops, or tablets) available to use during reading instruction?",
            "ATBR12BA": "What access do the students have to digital devices? The school provides each student with a digital device",
            "ATBR12BB": "What access do the students have to digital devices? The class has digital devices that students can share",
            "ATBR12BC": "What access do the students have to digital devices? The school has digital devices that the class can use sometimes",
            "ATBR12BD": "What access do the students have to digital devices? Students bring their own digital devices",
            "ATBR12C": "When doing reading activities with the whole class, how often do you have students use digital devices?",
            "ATBR12DA": "In addition, how often do you have specific groups of students do personalized reading activities using digital devices? Low-performing students",
            "ATBR12DB": "In addition, how often do you have specific groups of students do personalized reading activities using digital devices? High-performing students",
            "ATBR12DC": "In addition, how often do you have specific groups of students do personalized reading activities using digital devices? Students with special needs",
            "ATBR12EA": "During reading instruction, how often do you have students use digital devices to do the following? Read digital texts",
            "ATBR12EB": "During reading instruction, how often do you have students use digital devices to do the following? Look up facts and definitions",
            "ATBR12EC": "During reading instruction, how often do you have students use digital devices to do the following? Do a research project on a particular topic or problem",
            "ATBR12ED": "During reading instruction, how often do you have students use digital devices to do the following? Write stories or other texts",
            "ATBR12EE": "During reading instruction, how often do you have students use digital devices to do the following? Create a presentation or communication (e.g., video)",
            "ATBR13A": "Do you have a library or reading corner in your classroom?",
            "ATBR13B": "About how many books with different titles are in your classroom library?",
            "ATBR13C": "About how many magazines with different titles are in your classroom library?",
            "ATBR13D": "How often do you give the students in your class time to use the classroom library or reading corner?",
            "ATBR13E": "Can the students borrow books from the classroom library or reading corner to take home?",
            "ATBR14": "How often do you take or send the students to the <school or local library>?",
            "ATBR15": "How often do you assign reading as part of homework (for any subject)?",
            "ATBR16": "In general, how much time do you expect students to spend on homework involving reading (for any subject) each time you assign it?",
            "ATBR17A": "How often do you do the following with the reading homework assignments for this class? Correct assignments and give feedback to students",
            "ATBR17B": "How often do you do the following with the reading homework assignments for this class? Discuss the homework in class",
            "ATBR17C": "How often do you do the following with the reading homework assignments for this class? Monitor whether or not the homework was completed",
            "ATBR18A": "How much importance do you place on the following assessment strategies in reading? Observing students as they work",
            "ATBR18B": "How much importance do you place on the following assessment strategies in reading? Asking students to answer questions during class",
            "ATBR18C": "How much importance do you place on the following assessment strategies in reading? Short, regular written assessments (paper or digital)",
            "ATBR18D": "How much importance do you place on the following assessment strategies in reading? Longer tests (e.g., unit tests or exams)",
            "ATBR18E": "How much importance do you place on the following assessment strategies in reading? Long-term projects (e.g., reading logs)",
            "ATBR19": "About how often do students in this class take reading assessments on digital devices?",
        },
        "ast": {
            "ASRIBM01": "INT. READING SCALE BENCHMARK REACHED. Performed below the Low International Benchmark",
            "ASRIBM02": "INT. READING SCALE BENCHMARK REACHED. Performed at or above the Low International Benchmark, but below the Intermediate Benchmark",
            "ASRIBM03": "INT. READING SCALE BENCHMARK REACHED. Performed at or above the Intermediate International Benchmark, but below the High Benchmark",
            "ASRIBM04": "INT. READING SCALE BENCHMARK REACHED. Performed at or above the High International Benchmark, but below the Advanced Benchmark",
            "ASRIBM05": "INT. READING SCALE BENCHMARK REACHED. Performed at or above the Advanced International Benchmark",
        },
    }

In [483]:
def read_codebooks():
    cols = ["Variable", "Label", "Level", "Value Scheme Detailed", "Decimals", "Comment"]

    sheets_normal = ["ASAR5", "ASRR5", "ASGR5", "ASHR5", "ACGR5", "ATGR5", "ASTR5"]
    sheets_bridge = ["ASAA5", "ASRA5", "ASGA5", "ASHA5", "ACGA5", "ATGA5", "ASTA5"]

    restrict_use = "In restricted-use IDB only"

    def read_sheet_from_file(filename, sheet_name):
        df = pd.read_excel(f"{INPUT}/{filename}", sheet_name=sheet_name)[cols]
        return df[df["Comment"] != restrict_use].drop(columns=["Comment"])

    dfs_normal = {
        sheet_name[0:3].lower(): read_sheet_from_file("P21_Codebook.xlsx", sheet_name)
        for sheet_name in sheets_normal
    }
    dfs_bridge = {
        sheet_name[0:3].lower(): read_sheet_from_file("P21Br_Codebook.xlsx", sheet_name)
        for sheet_name in sheets_bridge
    }

    return {"normal": dfs_normal, "bridge": dfs_bridge}

In [301]:
def get_item_informations():
    sheets = ["P21 Paper", "P21 Digital", "P21Br (Paper)"]
    cols = ["Item ID", "Label"]

    dfs = [
        pd.read_excel(f"{INPUT}/item_information.xlsx", sheet_name=sheet_name)[cols]
        for sheet_name in sheets
    ]

    return pd.concat(dfs).drop_duplicates(subset=['Item ID'])

In [495]:
codebooks = read_codebooks()

In [303]:
item_informations = get_item_informations()

In [434]:
labels_from_context_questionnaires = gen_labels_from_context_questionnaires()

In [496]:
def gen_archs(codebooks: dict[str, dict[str, pd.DataFrame]]):
    def find_label(table, variable, label_original, label_from_item):
        if pd.isnull(label_from_item):
            if (
                table in labels_from_context_questionnaires
                and variable in labels_from_context_questionnaires[table]
            ):
                return labels_from_context_questionnaires[table][variable]
            else:
                return label_original
        else:
            return label_from_item

    def get_bigquery_type(variable, level, decimals, scheme):
        if variable in [
            "IDPOP",
            "IDGRADER",
            "IDGRADE",
            "WAVE",
            "IDSCHOOL",
            "IDCLASS",
            "IDSTUD",
            "ITSEX",
            "ITADMINI",
            "ITLANG_SA",
            "LCID_SA",
            "IDBOOK",
            "ITDEV",
            "VERSION",
            "SCOPE",
        ]:
            return "string"

        if not pd.isnull(scheme) and scheme.strip() == "1: Yes; 2: No":
            return "bool"

        if decimals > 0:
            assert pd.isnull(scheme)
            return "float64"
        else:
            return "int64" if level == "Scale" else "string"

    def process_table(df: pd.DataFrame, table: str):
        order = [
            "name",
            "bigquery_type",
            "description",
            "temporal_coverage",
            "covered_by_dictionary",
            "directory_column",
            "measurement_unit",
            "has_sensitive_data",
            "observations",
            "original_name",
        ]

        df["name"] = df["Variable"].apply(
            lambda var: RENAMES[table][var] if var in RENAMES[table] else var.lower()
        )
        df = df.merge(
            item_informations, how="left", left_on="Variable", right_on="Item ID"
        )

        df["description"] = df[["Variable", "Label_x", "Label_y"]].apply(
            lambda cols: find_label(table, cols[0], cols[1], cols[2]).title(), axis=1
        )

        df["bigquery_type"] = df[
            ["Variable", "Level", "Decimals", "Value Scheme Detailed"]
        ].apply(lambda cols: get_bigquery_type(*cols), axis=1)

        df["covered_by_dictionary"] = df["Value Scheme Detailed"].apply(
            lambda scheme: "no" if pd.isnull(scheme) else "yes"
        )

        df["observations"] = None

        df["directory_column"] = df["Variable"].apply(
            lambda var: "br_bd_diretorios_mundo.pais:id_pais_m49"
            if var == "IDCNTRY"
            else None
        )

        df["measurement_unit"] = df["Variable"].apply(
            lambda var: "person" if var == "NTEACH" else None
        )

        df["temporal_coverage"] = None
        df["has_sensitive_data"] = "no"

        df = df[df["Variable"] != "isDummy"]

        pirls_type_row = pd.DataFrame(
            [
                {
                    "name": "pirls_type",
                    "bigquery_type": "string",
                    "description": "Indicates if the record is from PIRLS Bridge or PIRLS Normal",
                    "temporal_coverage": None,
                    "covered_by_dictionary": "no",
                    "directory_column": None,
                    "measurement_unit": None,
                    "has_sensitive_data": "no",
                    "observations": None,
                    "original_name": None,
                }
            ]
        )

        df = (
            df.drop(columns=["Label_x", "Item ID", "Label_y", "Decimals"])
            .rename(columns={"Variable": "original_name"})
            .filter(order)
        )

        return pd.concat([df, pirls_type_row])

    # return process_table(codebooks["normal"]["asa"], "asa")

    result = dict(acg=[], asa=[], asg=[], ash=[], asr=[], ast=[], atg=[])

    for codebook in codebooks.keys():
        for table in codebooks[codebook].keys():
            df = codebooks[codebook][table]

            result[table].append(process_table(df, table))

    return {
        table: pd.concat(result[table]).drop_duplicates(subset=["name"])
        for table in result.keys()
    }

In [497]:
tables_archs = gen_archs(codebooks)

In [498]:
for table in tables_archs.keys():
    df = tables_archs[table]
    table_name = f"{CWD}/extra/architecture/{PIRLS_TABLES_DESC[table]}_{table}.xlsx"
    df.to_excel(table_name, index=False)

# Tratamento

In [501]:
def join_pirls(dfs):
    result = dict(acg=[], asa=[], asg=[], ash=[], asr=[], ast=[], atg=[])

    for kind in dfs.keys():
        for table in dfs[kind].keys():
            df: pd.DataFrame = dfs[kind][table]

            df["pirls_type"] = "Normal" if kind == "normal" else "Bridge"

            df.rename(columns=RENAMES[table])
            df.rename(columns=str.lower)

            result[table].append(df)

    return {table: pd.concat(result[table]) for table in result.keys()}